In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
from statsmodels.formula.api import ols
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Lasso
from math import sqrt
import pickle as pkl


plt.style.use('ggplot')
pd.set_option('display.max_columns', 500)

In [21]:
features = pkl.load(open('features.pkl','rb'))
m_feature = pkl.load(open('m_feature.pkl','rb'))
p_feature = pkl.load(open('p_feature.pkl','rb'))


# original data set for math class
m_orig = pd.read_csv('MathClassOriginal.csv')
m_orig = m_orig.drop('Unnamed: 0',axis =1)

# original dataset for language arts class
p_orig = pd.read_csv('PortugeseClassOriginal.csv')
p_orig = p_orig.drop('Unnamed: 0',axis =1)

m = pd.read_csv('MathClassCleaned.csv')
m = m.drop(columns='Unnamed: 0')
p = pd.read_csv('PortugeseClassCleaned.csv')
p = p.drop(columns='Unnamed: 0')
p_target=p.G3
p_data = p_feature
p_data = p_data.drop(columns = ['school',"guardian"])

In [16]:
model = ols(data = p, formula='G3~studytime+failures+schoolsup+higher+freetime+Dalc+Walc+health+address_type_Urban+Pstatus_Together+Mjob_health+Mjob_services+Mjob_teacher+Fjob_health+Fjob_services+reason_course+reason_home+reason_reputation').fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     G3   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.290
Method:                 Least Squares   F-statistic:                     15.71
Date:                Sun, 13 Sep 2020   Prob (F-statistic):           4.67e-40
Time:                        01:43:53   Log-Likelihood:                -1561.2
No. Observations:                 649   AIC:                             3160.
Df Residuals:                     630   BIC:                             3245.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              9.9124      0.758     13.078      0.000       8.424      11.401
studytime              0.5108      0.138      3.709      0.000       0.240       0.781
failures              -1.4916      0.195     -7.664      0.000      -1.874      -1.109
schoolsup             -1.0429      0.357     -2.920      0.004      -1.744      -0.341
higher                 1.8288      0.378      4.837      0.000       1.086       2.571
freetime              -0.1685      0.105     -1.612      0.107      -0.374       0.037
Dalc                  -0.2269      0.151     -1.503      0.133      -0.523       0.070
Walc                  -0.1526      0.110     -1.393      0.164      -0.368       0.062
health                -0.1657      0.077     -2.166      0.031      -0.316      -0.015
address_type_Urban     0.7055      0.242      2.913      0.004       0.230       1.181
Pstatus_Together       0.2435      0.330      0.737      0.461      -0.405       0.892
Mjob_health            1.1107      0.444      2.504      0.013       0.240       1.982
Mjob_services          0.6261      0.283      2.213      0.027       0.070       1.182
Mjob_teacher           1.0041      0.361      2.785      0.006       0.296       1.712
Fjob_health           -0.4820      0.615     -0.784      0.433      -1.690       0.726
Fjob_services         -0.5802      0.251     -2.315      0.021      -1.072      -0.088
reason_course          0.7165      0.367      1.951      0.051      -0.005       1.438
reason_home            0.8107      0.400      2.024      0.043       0.024       1.597
reason_reputation      1.2379      0.406      3.045      0.002       0.440       2.036
==============================================================================
Omnibus:                      132.762   Durbin-Watson:                   1.867
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              427.221
Skew:                          -0.960   Prob(JB):                     1.70e-93
Kurtosis:                       6.481   Cond. No.                         49.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    p_data, target, random_state=9,test_size=0.2
)
scaler = StandardScaler()

# fit the scaler to the training data
scaler.fit(X_train)

#transform the training data
X_train = pd.DataFrame(data=scaler.transform(X_train), columns=p_data.columns)

#transform the testing dat
X_test = pd.DataFrame(data=scaler.transform(X_test), columns=p_data.columns)

#instantiate a linear regression object
lm = LinearRegression()

#fit the linear regression to the data
lm = lm.fit(X_train, y_train)

y_train_pred = lm.predict(X_train)

train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))


print('Training Root Mean Squared Error:' , train_rmse)




Training Root Mean Squared Error: 2.691212447987795


In [26]:
selector = SelectKBest(f_regression, k=15)

selector.fit(X_train, y_train)

selected_columns = X_train.columns[selector.get_support()]
removed_columns = X_train.columns[~selector.get_support()]

In [27]:
len(removed_columns)

20

In [28]:
#instantiate a linear regression object
lm_kbest = LinearRegression()

#fit the linear regression to the data
lm_kbest = lm_kbest.fit(X_train[selected_columns], y_train)

y_train_kbest = lm_kbest.predict(X_train[selected_columns])


trainK_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_kbest))


print('Training Root Mean Squared Error:' , trainK_rmse)

y_kbest = lm_kbest.predict(X_test[selected_columns])

testK_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_kbest))

print('Testing Root Mean Squared Error:' , testK_rmse)


Training Root Mean Squared Error: 2.769321851114437
Testing Root Mean Squared Error: 2.5818889317860414


In [29]:
#instantiate a linear regression object
lm = LinearRegression()

#fit the linear regression to the data
lm = lm.fit(X_train, y_train)

y_train_pred = lm.predict(X_train)

train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))


print('Training Root Mean Squared Error:' , train_rmse)

Training Root Mean Squared Error: 2.691212447987795


In [30]:
#use fitted model to predict on the test examples
y_test_pred = lm.predict(X_test)

#evaluate the predictions on the test examples
test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

print('Testing Root Mean Squared Error:' , test_rmse)


print('Training: ', (train_rmse), "vs. Testing: ", (test_rmse))

Testing Root Mean Squared Error: 2.5818025560884137
Training:  2.691212447987795 vs. Testing:  2.5818025560884137


In [32]:
selector = SelectKBest(f_regression, k=20)

selector.fit(X_train, y_train)

selected_columns = X_train.columns[selector.get_support()]
removed_columns = X_train.columns[~selector.get_support()]

In [33]:
list(selected_columns)

['Medu',
 'Fedu',
 'traveltime',
 'studytime',
 'failures',
 'higher',
 'internet',
 'romantic',
 'freetime',
 'Dalc',
 'Walc',
 'health',
 'absences',
 'address_type_Urban',
 'Mjob_at_home',
 'Mjob_teacher',
 'Fjob_teacher',
 'reason_course',
 'reason_reputation',
 'super']

In [34]:
list(removed_columns)

['schoolsup',
 'famsup',
 'activities',
 'nursery',
 'famrel',
 'goout',
 'famsize_BIG_FAMILY',
 'Pstatus_Together',
 'Mjob_health',
 'Mjob_services',
 'Fjob_at_home',
 'Fjob_health',
 'Fjob_services',
 'reason_home',
 'tutor']

In [35]:
#instantiate a linear regression object
lm_kbest = LinearRegression()

#fit the linear regression to the data
lm_kbest = lm_kbest.fit(X_train[selected_columns], y_train)

y_train_kbest = lm_kbest.predict(X_train[selected_columns])


trainK_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_kbest))


print('Training Root Mean Squared Error:' , trainK_rmse)

y_kbest = lm_kbest.predict(X_test[selected_columns])

testK_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_kbest))

print('Testing Root Mean Squared Error:' , testK_rmse)




Training Root Mean Squared Error: 2.7493255562626446
Testing Root Mean Squared Error: 2.57145840237804


In [36]:
print('Original: ', test_rmse, '\n',
      "KBest:   ", testK_rmse,'\n')

Original:  2.5818025560884137 
 KBest:    2.57145840237804 



In [40]:
estimator = linear_model.LinearRegression()
# Create recursive feature eliminator that scores features by mean squared errors
selector = RFECV(estimator,step=1, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)

# Fit recursive feature eliminator 
selector.fit(X_train, y_train)


RFECV(cv=5, estimator=LinearRegression(), n_jobs=-1,
      scoring='neg_mean_squared_error', verbose=1)

In [41]:
selected_rfe = X_train.columns[selector.support_]
removed_rfe = X_train.columns[~selector.support_]


In [42]:
list(removed_rfe)

[]

In [43]:
len(list(selected_rfe))

35

In [44]:
#instantiate a linear regression object
lm_rfe = LinearRegression()

#fit the linear regression to the data
lm_rfe = lm_rfe.fit(X_train[selected_rfe], y_train)

y_rfe = lm_rfe.predict(X_train[selected_rfe])


trainRFE_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_rfe))


print('Training Root Mean Squared Error:' , trainRFE_rmse)

y_pred_rfe = lm_rfe.predict(X_test[selected_rfe])

testRFE_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred_rfe))

print('Testing Root Mean Squared Error:' , testRFE_rmse)

Training Root Mean Squared Error: 2.691212447987795
Testing Root Mean Squared Error: 2.5818025560884137


In [45]:
print('Original: ', test_rmse, '\n',
      "KBest:   ", testK_rmse,'\n',
      "RFE:     ", testRFE_rmse)

Original:  2.5818025560884137 
 KBest:    2.57145840237804 
 RFE:      2.5818025560884137


In [50]:
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly2_data = poly_2.fit_transform(p_data)
poly2_columns = poly_2.get_feature_names(p_data.columns)
df_poly2 = pd.DataFrame(poly2_data, columns=poly2_columns)
df_poly2.head()

,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,address_type_Urban,famsize_BIG_FAMILY,Pstatus_Together,Mjob_at_home,Mjob_health,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_reputation,tutor,super,Medu^2,Medu Fedu,Medu traveltime,Medu studytime,Medu failures,Medu schoolsup,Medu famsup,Medu activities,Medu nursery,Medu higher,Medu internet,Medu romantic,Medu famrel,Medu freetime,Medu goout,Medu Dalc,Medu Walc,Medu health,Medu absences,Medu address_type_Urban,Medu famsize_BIG_FAMILY,Medu Pstatus_Together,Medu Mjob_at_home,Medu Mjob_health,Medu Mjob_services,Medu Mjob_teacher,Medu Fjob_at_home,Medu Fjob_health,Medu Fjob_services,Medu Fjob_teacher,Medu reason_course,Medu reason_home,Medu reason_reputation,Medu tutor,Medu super,Fedu^2,Fedu traveltime,Fedu studytime,Fedu failures,Fedu schoolsup,Fedu famsup,Fedu activities,Fedu nursery,Fedu higher,Fedu internet,Fedu romantic,Fedu famrel,Fedu freetime,Fedu goout,Fedu Dalc,Fedu Walc,Fedu health,Fedu absences,Fedu address_type_Urban,Fedu famsize_BIG_FAMILY,Fedu Pstatus_Together,Fedu Mjob_at_home,Fedu Mjob_health,Fedu Mjob_services,Fedu Mjob_teacher,Fedu Fjob_at_home,Fedu Fjob_health,Fedu Fjob_services,Fedu Fjob_teacher,Fedu reason_course,Fedu reason_home,Fedu reason_reputation,Fedu tutor,Fedu super,traveltime^2,traveltime studytime,traveltime failures,traveltime schoolsup,traveltime famsup,traveltime activities,traveltime nursery,traveltime higher,traveltime internet,traveltime romantic,traveltime famrel,traveltime freetime,traveltime goout,traveltime Dalc,traveltime Walc,traveltime health,traveltime absences,traveltime address_type_Urban,traveltime famsize_BIG_FAMILY,traveltime Pstatus_Together,traveltime Mjob_at_home,traveltime Mjob_health,traveltime Mjob_services,traveltime Mjob_teacher,traveltime Fjob_at_home,traveltime Fjob_health,traveltime Fjob_services,traveltime Fjob_teacher,traveltime reason_course,traveltime reason_home,traveltime reason_reputation,traveltime tutor,traveltime super,studytime^2,studytime failures,studytime schoolsup,studytime famsup,studytime activities,studytime nursery,studytime higher,studytime internet,studytime romantic,studytime famrel,studytime freetime,studytime goout,studytime Dalc,studytime Walc,studytime health,studytime absences,studytime address_type_Urban,studytime famsize_BIG_FAMILY,studytime Pstatus_Together,studytime Mjob_at_home,studytime Mjob_health,studytime Mjob_services,studytime Mjob_teacher,studytime Fjob_at_home,studytime Fjob_health,studytime Fjob_services,studytime Fjob_teacher,studytime reason_course,studytime reason_home,studytime reason_reputation,studytime tutor,studytime super,failures^2,failures schoolsup,failures famsup,failures activities,failures nursery,failures higher,failures internet,failures romantic,failures famrel,failures freetime,failures goout,failures Dalc,failures Walc,failures health,failures absences,failures address_type_Urban,failures famsize_BIG_FAMILY,failures Pstatus_Together,failures Mjob_at_home,failures Mjob_health,failures Mjob_services,failures Mjob_teacher,failures Fjob_at_home,failures Fjob_health,failures Fjob_services,failures Fjob_teacher,failures reason_course,failures reason_home,failures reason_reputation,failures tutor,failures super,schoolsup^2,schoolsup famsup,schoolsup activities,schoolsup nursery,schoolsup higher,schoolsup internet,schoolsup romantic,schoolsup famrel,schoolsup freetime,schoolsup goout,schoolsup Dalc,schoolsup Walc,schoolsup health,schoolsup absences,schoolsup address_type_Urban,schoolsup famsize_BIG_FAMILY,schoolsup Pstatus_Together,schoolsup Mjob_at_home,schoolsup Mjob_health,schoolsup Mjob_services,schoolsup Mjob_teacher,schoolsup Fjob_at_home,schoolsup Fjob_health,schoolsup Fjob_services,schoolsup Fjob_teacher,schoolsup reason_course,schoolsup reason_home,schoolsup reason_reputation,schoolsup tutor,schoo

In [51]:
X_train, X_test, y_train, y_test = train_test_split(
    df_poly2, target, random_state=9,test_size=0.2
         )
scaler2 = StandardScaler()
# fit the scaler to the training data
scaler2.fit(X_train)

#transform the training data
X_train = pd.DataFrame(data=scaler2.transform(X_train), columns=df_poly2.columns)

#transform the testing dat
X_test = pd.DataFrame(data=scaler2.transform(X_test), columns=df_poly2.columns)


In [52]:
# instantiate a linear regression object
lm_2 = LinearRegression()

#fit the linear regression to the data
lm_2 = lm_2.fit(X_train, y_train)

y_train_pred = lm_2.predict(X_train)

train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))


print('Training Root Mean Squared Error:' , train_rmse)

Training Root Mean Squared Error: 8.342648506431632e-14


In [53]:
# use fitted model to predict on test data
y_pred = lm_2.predict(X_test)

test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

print('Testing Root Mean Squared Error:' , test_rmse)


print('Training: ', int(train_rmse), "vs. Testing: ", int(test_rmse))

Testing Root Mean Squared Error: 22.694844375509433
Training:  0 vs. Testing:  22


In [54]:
selector = SelectKBest(f_regression, k=80)

selector.fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


SelectKBest(k=80, score_func=<function f_regression at 0x7fbca0b4e9d0>)

In [55]:
selected_columns = X_train.columns[selector.get_support()]
removed_columns = X_train.columns[~selector.get_support()]

In [56]:
list(selected_columns)

['Medu',
 'Fedu',
 'studytime',
 'failures',
 'higher',
 'Dalc',
 'Walc',
 'super',
 'Medu^2',
 'Medu Fedu',
 'Medu studytime',
 'Medu failures',
 'Medu higher',
 'Medu internet',
 'Medu address_type_Urban',
 'Medu super',
 'Fedu^2',
 'Fedu studytime',
 'Fedu failures',
 'Fedu higher',
 'Fedu internet',
 'Fedu famrel',
 'Fedu address_type_Urban',
 'Fedu reason_reputation',
 'Fedu super',
 'traveltime failures',
 'studytime^2',
 'studytime failures',
 'studytime higher',
 'studytime internet',
 'studytime famrel',
 'studytime address_type_Urban',
 'studytime reason_reputation',
 'studytime super',
 'failures^2',
 'failures famsup',
 'failures activities',
 'failures nursery',
 'failures higher',
 'failures internet',
 'failures romantic',
 'failures famrel',
 'failures freetime',
 'failures goout',
 'failures Dalc',
 'failures Walc',
 'failures health',
 'failures absences',
 'failures address_type_Urban',
 'failures famsize_BIG_FAMILY',
 'failures Pstatus_Together',
 'failures Mjob_at_

In [57]:
#instantiate a linear regression object
lm_kbest = LinearRegression()

#fit the linear regression to the data
lm_kbest = lm_kbest.fit(X_train[selected_columns], y_train)

y_train_kbest = lm_kbest.predict(X_train[selected_columns])


trainK_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_kbest))


print('Training Root Mean Squared Error:' , trainK_rmse)

y_kbest = lm_kbest.predict(X_test[selected_columns])

testK_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_kbest))

print('Testing Root Mean Squared Error:' , testK_rmse)




Training Root Mean Squared Error: 2.911824433765815
Testing Root Mean Squared Error: 2.897913109569523


In [84]:
poly_5 = PolynomialFeatures(degree=5, include_bias=False)
poly5_data = poly_5.fit_transform(p_data)
poly5_columns = poly_5.get_feature_names(p_data.columns)
df_poly5 = pd.DataFrame(poly5_data, columns=poly5_columns)
df_poly5.columns = df_poly5.columns.str.replace(' ', '')
df_poly5.columns = df_poly5.columns.str.replace('^', '')
df_poly5.head()

,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,address_type_Urban,famsize_BIG_FAMILY,Pstatus_Together,Mjob_at_home,Mjob_health,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_reputation,tutor,super,Medu2,MeduFedu,Medutraveltime,Medustudytime,Medufailures,Meduschoolsup,Medufamsup,Meduactivities,Medunursery,Meduhigher,Meduinternet,Meduromantic,Medufamrel,Medufreetime,Medugoout,MeduDalc,MeduWalc,Meduhealth,Meduabsences,Meduaddress_type_Urban,Medufamsize_BIG_FAMILY,MeduPstatus_Together,MeduMjob_at_home,MeduMjob_health,MeduMjob_services,MeduMjob_teacher,MeduFjob_at_home,MeduFjob_health,MeduFjob_services,MeduFjob_teacher,Medureason_course,Medureason_home,Medureason_reputation,Medututor,Medusuper,Fedu2,Fedutraveltime,Fedustudytime,Fedufailures,Feduschoolsup,Fedufamsup,Feduactivities,Fedunursery,Feduhigher,Feduinternet,Feduromantic,Fedufamrel,Fedufreetime,Fedugoout,FeduDalc,FeduWalc,Feduhealth,Feduabsences,Feduaddress_type_Urban,Fedufamsize_BIG_FAMILY,FeduPstatus_Together,FeduMjob_at_home,FeduMjob_health,FeduMjob_services,FeduMjob_teacher,FeduFjob_at_home,FeduFjob_health,FeduFjob_services,FeduFjob_teacher,Fedureason_course,Fedureason_home,Fedureason_reputation,Fedututor,Fedusuper,traveltime2,traveltimestudytime,traveltimefailures,traveltimeschoolsup,traveltimefamsup,traveltimeactivities,traveltimenursery,traveltimehigher,traveltimeinternet,traveltimeromantic,traveltimefamrel,traveltimefreetime,traveltimegoout,traveltimeDalc,traveltimeWalc,traveltimehealth,traveltimeabsences,traveltimeaddress_type_Urban,traveltimefamsize_BIG_FAMILY,traveltimePstatus_Together,traveltimeMjob_at_home,traveltimeMjob_health,traveltimeMjob_services,traveltimeMjob_teacher,traveltimeFjob_at_home,traveltimeFjob_health,traveltimeFjob_services,traveltimeFjob_teacher,traveltimereason_course,traveltimereason_home,traveltimereason_reputation,traveltimetutor,traveltimesuper,studytime2,studytimefailures,studytimeschoolsup,studytimefamsup,studytimeactivities,studytimenursery,studytimehigher,studytimeinternet,studytimeromantic,studytimefamrel,studytimefreetime,studytimegoout,studytimeDalc,studytimeWalc,studytimehealth,studytimeabsences,studytimeaddress_type_Urban,studytimefamsize_BIG_FAMILY,studytimePstatus_Together,studytimeMjob_at_home,studytimeMjob_health,studytimeMjob_services,studytimeMjob_teacher,studytimeFjob_at_home,studytimeFjob_health,studytimeFjob_services,studytimeFjob_teacher,studytimereason_course,studytimereason_home,studytimereason_reputation,studytimetutor,studytimesuper,failures2,failuresschoolsup,failuresfamsup,failuresactivities,failuresnursery,failureshigher,failuresinternet,failuresromantic,failuresfamrel,failuresfreetime,failuresgoout,failuresDalc,failuresWalc,failureshealth,failuresabsences,failuresaddress_type_Urban,failuresfamsize_BIG_FAMILY,failuresPstatus_Together,failuresMjob_at_home,failuresMjob_health,failuresMjob_services,failuresMjob_teacher,failuresFjob_at_home,failuresFjob_health,failuresFjob_services,failuresFjob_teacher,failuresreason_course,failuresreason_home,failuresreason_reputation,failurestutor,failuressuper,schoolsup2,schoolsupfamsup,schoolsupactivities,schoolsupnursery,schoolsuphigher,schoolsupinternet,schoolsupromantic,schoolsupfamrel,schoolsupfreetime,schoolsupgoout,schoolsupDalc,schoolsupWalc,schoolsuphealth,schoolsupabsences,schoolsupaddress_type_Urban,schoolsupfamsize_BIG_FAMILY,schoolsupPstatus_Together,schoolsupMjob_at_home,schoolsupMjob_health,schoolsupMjob_services,schoolsupMjob_teacher,schoolsupFjob_at_home,schoolsupFjob_health,schoolsupFjob_services,schoolsupFjob_teacher,schoolsupreason_course,schoolsupreason_home,schoolsupreason_reputation,schoolsuptutor,schoolsupsuper,famsup2,famsupactivities,famsupnursery,famsuphigher,famsupinternet,famsupromantic,famsupfamrel,famsupfreetime,famsupgoout,famsupDalc,famsupWalc,famsuphealth,famsupabsences,famsupaddres

In [91]:
X_train, X_test, y_train, y_test = train_test_split(
    df_poly5, target, random_state=9,test_size=0.2
         )
scaler2 = StandardScaler()
# fit the scaler to the training data
scaler2.fit(X_train)

#transform the training data
X_train = pd.DataFrame(data=scaler2.transform(X_train), columns=df_poly5.columns)

#transform the testing dat
X_test = pd.DataFrame(data=scaler2.transform(X_test), columns=df_poly5.columns)


In [92]:
# instantiate a linear regression object
lm_5 = LinearRegression()

#fit the linear regression to the data
lm_5 = lm_5.fit(X_train, y_train)

y_train_pred = lm_5.predict(X_train)

train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))


print('Training Root Mean Squared Error:' , train_rmse)

Training Root Mean Squared Error: 2.382274783466104e-12


In [103]:
selector = SelectKBest(f_regression, k=30)

selector.fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


SelectKBest(k=30, score_func=<function f_regression at 0x7fbca0b4e9d0>)

In [104]:
selected_columns = X_train.columns[selector.get_support()]
removed_columns = X_train.columns[~selector.get_support()]

In [105]:
ols_selected_cols = list(selected_columns)

In [106]:
#instantiate a linear regression object
lm_kbest = LinearRegression()

#fit the linear regression to the data
lm_kbest = lm_kbest.fit(X_train[selected_columns], y_train)

y_train_kbest = lm_kbest.predict(X_train[selected_columns])


trainK_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_kbest))


print('Training Root Mean Squared Error:' , trainK_rmse)

y_kbest = lm_kbest.predict(X_test[selected_columns])

testK_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_kbest))

print('Testing Root Mean Squared Error:' , testK_rmse)




Training Root Mean Squared Error: 2.812494190922182
Testing Root Mean Squared Error: 2.6419146587113875


In [107]:
'+'.join(ols_selected_cols)

'failures+studytimehigher+failuresnursery+failuresfamrel+failuresfreetime+failuresgoout+failuresWalc+failuresfamsize_BIG_FAMILY+studytimehigher2+failuresnursery2+failuresnurserygoout+failuresfreetimegoout+failuresfreetimeWalc+failuresfamsize_BIG_FAMILY2+studytimehigher3+studytimehigherinternetaddress_type_Urban+failuresnursery3+failuresnursery2goout+failuresfreetime2goout+failuresfreetime2Walc+failuresfreetimegooutWalc+failuresfamsize_BIG_FAMILY3+traveltimefailuresfreetime2Walc+studytimehigher4+studytimehigher2internetaddress_type_Urban+studytimehigherinternet2address_type_Urban+studytimehigherinternetaddress_type_Urban2+failuresnursery4+failuresnursery3goout+failuresfamsize_BIG_FAMILY4'

In [109]:
ols_selected_cols
X_train['g3'] = p.G3
crazy_line = ols(data = X_train, formula = 'g3~failures+studytimehigher+failuresnursery+failuresfamrel+failuresfreetime+failuresgoout+failuresWalc+failuresfamsize_BIG_FAMILY+studytimehigher2+failuresnursery2+failuresnurserygoout+failuresfreetimegoout+failuresfreetimeWalc+failuresfamsize_BIG_FAMILY2+studytimehigher3+studytimehigherinternetaddress_type_Urban+failuresnursery3+failuresnursery2goout+failuresfreetime2goout+failuresfreetime2Walc+failuresfreetimegooutWalc+failuresfamsize_BIG_FAMILY3+traveltimefailuresfreetime2Walc+studytimehigher4+studytimehigher2internetaddress_type_Urban+studytimehigherinternet2address_type_Urban+studytimehigherinternetaddress_type_Urban2+failuresnursery4+failuresnursery3goout+failuresfamsize_BIG_FAMILY4').fit()
crazy_line.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     g3   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.7052
Date:                Sun, 13 Sep 2020   Prob (F-statistic):              0.790
Time:                        03:01:02   Log-Likelihood:                -1250.2
No. Observations:                 519   AIC:                             2534.
Df Residuals:                     502   BIC:                             2607.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
Intercept                                     12.2987      0.120    102.400      0.000      12.063      12.535
failures                                       1.3809      1.234      1.119      0.264      -1.044       3.805
studytimehigher                                0.0161      0.037      0.438      0.662      -0.056       0.088
failuresnursery                               -0.1504      0.177     -0.847      0.397      -0.499       0.198
failuresfamrel                                -0.4219      0.559     -0.755      0.451      -1.520       0.676
failuresfreetime                              -1.4578      1.078     -1.353      0.177      -3.575       0.660
failuresgoout                                 -3.0913      3.081     -1.003      0.316      -9.145       2.962
failuresWalc                                   2.3956      3.088      0.776      0.438      -3.671       8.462
failuresfamsize_BIG_FAMILY                     0.0724      0.074      0.985      0.325      -0.072       0.217
studytimehigher2                               0.0161      0.037      0.438      0.662      -0.056       0.088
failuresnursery2                              -0.1504      0.177     -0.847      0.397      -0.499       0.198
failuresnurserygoout                           0.1059      0.234      0.452      0.652      -0.355       0.567
failuresfreetimegoout                          8.0999      6.504      1.245      0.214      -4.679      20.879
failuresfreetimeWalc                          -6.1210      6.674     -0.917      0.359     -19.232       6.991
failuresfamsize_BIG_FAMILY2                    0.0724      0.074      0.985      0.325      -0.072       0.217
studytimehigher3                               0.0161      0.037      0.438      0.662      -0.056       0.088
studytimehigherinternetaddress_type_Urban     -0.0455      0.036     -1.260      0.208      -0.116       0.025
failuresnursery3                              -0.1504      0.177     -0.847      0.397      -0.499       0.198
failuresnursery2goout                          0.1059      0.234      0.452      0.652      -0.355       0.567
failuresfreetime2goout                        -5.0440      3.992     -1.264      0.207     -12.887       2.799
failuresfreetime2Walc                          5.1605      4.181      1.234      0.218      -3.053      13.374
failuresfreetimegooutWalc                     -0.5727      1.196     -0.479      0.632      -2.923       1.778
failuresfamsize_BIG_FAMILY3                    0.0724      0.074      0.985      0.325      -0.072       0.217
traveltimefailuresfreetime2Walc               -0.5156      0.381     -1.355      0.176      -1.263       0.232
studytimehigher4                               0.0161      0.037      0.438      0.662

In [98]:
crazy_line.summary()

NameError: name 'crazy_line' is not defined

ERROR! Session/line number was not unique in database. History logging moved to new session 175
